In [6]:
class NPP_timeseries:
    '''
    class NPP_TotalGlobal(resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',savefig=False,mask="Global Ocean")
                 
    Output:
    self.NPPtotal [Pg C/year]
    self.PhyTotal [Pg C/year]
    self.DiaTotal [Pg C/year]
    self.EPtotal [Pg C/yr]
    self.SiEtotal [Tmol Si/yr]
    
    '''
    def __init__(self,resultpath,savepath,mesh,first_year,last_year,
                 mapproj='pc',
                 plotting = True,
                 savefig=False,
                 mask = "Global Ocean",
                 output = False,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.plotting = plotting
        self.savefig = savefig
        self.mask = mask

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        import pyfesom2 as pf
        from pathlib import Path
        # load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        
        # load nodal area -------------------------------------------------------------------------------------
        meshdiag = pf.get_meshdiag(mesh)
        nod_area_surface = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        
        # get depth of export production
        i_ep_depth = pf.ind_for_depth(100,mesh)
        nod_depth = meshdiag.zbar_n_bottom
        ep_depth = nod_depth[i_ep_depth]
        #print('EP for selected depth = {0} m'.format(ep_depth))
        #print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
        
        # check if coccos, second det, and variable sinking velocity are used or not -------------------------------------------------------------------------------
        cocco_path  = Path(self.resultpath + '/NPPc.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path  = Path(self.resultpath + '/NPPp.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        secdet_path = Path(self.resultpath + '/idetz2c.fesom.'+str(years[0])+'.nc')
        cram_path   = Path(self.resultpath + '/wsink_det1.fesom.'+str(years[0])+'.nc')
        
        # calculating total NPP per year -------------------------------------------------------------------------------
            
        NPPn = pf.get_data(resultpath, "NPPn", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        NPPd = pf.get_data(resultpath, "NPPd", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        if cocco_path.is_file():
            NPPc = pf.get_data(resultpath, "NPPc", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            print('Coccolithophores are used')
        if phaeo_path.is_file():
            NPPp = pf.get_data(resultpath, "NPPp", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            print('Phaeocystis is used')
        DetC1 = pf.get_data(resultpath, "DetC", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        SiE1 = pf.get_data(resultpath, "DetSi", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        DetCalc1 = pf.get_data(resultpath, "DetCalc", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        if secdet_path.is_file():
            DetC2 = pf.get_data(resultpath, "idetz2c", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            SiE2 = pf.get_data(resultpath, "idetz2si", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            DetCalc2 = pf.get_data(resultpath, "idetz2calc", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            print('Second detritus group is used')
        
        ## NPPn:units = "mmolC/(m2*d)"
        
        NPPn = NPPn.resample(time='YS').mean(dim='time').compute()
        NPPd = NPPd.resample(time='YS').mean(dim='time').compute()
        if cocco_path.is_file():
            NPPc = NPPc.resample(time='YS').mean(dim='time').compute()
        if phaeo_path.is_file():
            NPPp = NPPp.resample(time='YS').mean(dim='time').compute()
        SiE1 = SiE1.resample(time='YS').mean(dim='time').compute() 
        DetCalc1 = DetCalc1.resample(time='YS').mean(dim='time').compute()
        DetC1 = DetC1.resample(time='YS').mean(dim='time').compute()
        if secdet_path.is_file():
            DetC2 = DetC2.resample(time='YS').mean(dim='time').compute()
            SiE2 = SiE2.resample(time='YS').mean(dim='time').compute()
            DetCalc2 = DetCalc2.resample(time='YS').mean(dim='time').compute()
        
        # can provide mask or mask name
        if isinstance(mask, str):
            maskstr = mask
            mask = pf.get_mask(mesh, mask)
        else:
            maskstr = mask
            mask = mask
        
        ## Primary Production = "[Pg C/year]"  
        if cocco_path.is_file() & phaeo_path.is_file():
            NPP    = 365* (NPPd+NPPn+NPPc+NPPp)*12.01 /1e18 # Conversion from [mg/m2/day]   => [mg/m2/yr] => [Pg C/year]
            NPPc   = 365* (NPPc)*12.01 /1e18
            NPPp   = 365* (NPPp)*12.01 /1e18
        
        elif cocco_path.is_file() and not phaeo_path.is_file():
            NPP    = 365* (NPPd+NPPn+NPPc)*12.01 /1e18 # Conversion from [mg/m2/day]   => [mg/m2/yr] => [Pg C/year]
            NPPc   = 365* (NPPc)*12.01 /1e18
        
        
        else:
            NPP    = 365* (NPPd+NPPn)*12.01 /1e18 # Conversion from [mg/m2/day]   => [mg/m2/yr] => [Pg C/year]
        NPPd       = 365* (NPPd)*12.01 /1e18 
        NPPn       = 365* (NPPn)*12.01 /1e18 
        
        NPP_timeseries = pf.areasum_data(NPP,mesh,mask)
        NPPd_timeseries = pf.areasum_data(NPPd,mesh,mask)
        NPPn_timeseries = pf.areasum_data(NPPn,mesh,mask)
        if cocco_path.is_file():
            NPPc_timeseries = pf.areasum_data(NPPc,mesh,mask)

        if phaeo_path.is_file():
            NPPp_timeseries = pf.areasum_data(NPPp,mesh,mask)
        
        del NPP, NPPd, NPPn
        if cocco_path.is_file():
            del NPPc
        if phaeo_path.is_file():
            del NPPp
            
        secperyear = 31536000 # = 86400 sec/day * 365 days; necessary to compute yearly fluxes with sinking velocity in m/s
        
        ## Carbon export, DetC:units = "[mmol/m3]"
        if cram_path.is_file(): 
            Vdet1 = pf.get_data(resultpath, "wsink_det1", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
            Vdet1 = Vdet1.resample(time='YS').mean(dim='time').compute()
            print('Cram parameterization with variable sinking velocities is used')
            if secdet_path.is_file():
                Vdet2 = pf.get_data(resultpath, "wsink_det2", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
                Vdet2 = Vdet2.resample(time='YS').mean(dim='time').compute()
        else:
            Vdet1 = 0.0288 * 100. + 20. ## sinking velocity
            if secdet_path.is_file():
                Vdet2 = 200. ## sinking velocity
                        
        if cram_path.is_file():
            print('shape DetC1: ',np.shape(DetC1))
            print('shape Vdet1: ',np.shape(Vdet1))
            detc1 = secperyear  * np.squeeze(DetC1[:,:,i_ep_depth]) * 12.01 * np.squeeze(-Vdet1[:,:,i_ep_depth]) /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]; wsink_det1 is a negative number
            if secdet_path.is_file():
                detc2 = secperyear  * np.squeeze(DetC2[:,:,i_ep_depth]) * 12.01 * np.squeeze(-Vdet2[:,:,i_ep_depth]) /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]; wsink_det2 is a negative number
        else:
            detc1 = 365 * DetC1[:,:,i_ep_depth] * 12.01 * Vdet1 /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
            if secdet_path.is_file():
                detc2 = 365 *DetC2[:,:,i_ep_depth] * 12.01 * Vdet2 /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
        
        detct = detc1
        if secdet_path.is_file():
            print('shape detct: ',np.shape(detct))
            print('shape detct2: ',np.shape(detct))
            detct = detct + detc2
        
        EP_timeseries = pf.areasum_data(detct,mesh,mask)
        
        del detc1, detct
        if secdet_path.is_file():
            del detc2
        
        ## Si export, DetSi:units = "[mmol/m3]"
        #SiE = 365. * SiE *28.085 * Vdet /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
        if cram_path.is_file():
            Sie1 =  secperyear * np.squeeze(SiE1[:,:,i_ep_depth]) * np.squeeze(-Vdet1[:,:,i_ep_depth]) /1e15 # [mmol/m3] => [mmol/m2/yr]  =>  [Tmol Si/yr]; wsink_det1 is a negative number
            if secdet_path.is_file():
                Sie2 = secperyear * np.squeeze(SiE2[:,:,i_ep_depth]) * np.squeeze(-Vdet2[:,:,i_ep_depth]) /1e15 # [mmol/m3] => [mmol/m2/yr]  =>  [Tmol Si/yr]; wsink_det2 is a negative number
        else:
            Sie1 = 365 * SiE1[:,:,i_ep_depth] * Vdet1 /1e15 # [mmol/m3] => [mmol/m2/yr]  =>  [Tmol Si/yr]      
            if secdet_path.is_file():
                Sie2 = 365 * SiE2[:,:,i_ep_depth] * Vdet2 /1e15 # [mmol/m3] => [mmol/m2/yr]  =>  [Tmol Si/yr]   
        
        siect = Sie1
        if secdet_path.is_file():
            siect = siect + Sie2
        
        SiE_timeseries = pf.areasum_data(siect,mesh,mask)   
        
        del Sie1, siect
        if secdet_path.is_file():
            del Sie2
        
        if cram_path.is_file():
            Detcalc1 = secperyear * np.squeeze(DetCalc1[:,:,i_ep_depth]) * 12.01 * np.squeeze(-Vdet1[:,:,i_ep_depth]) /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]; wsink_det1 is a negative number
            if secdet_path.is_file():
                Detcalc2 = secperyear * np.squeeze(DetCalc2[:,:,i_ep_depth]) * 12.01 * np.squeeze(-Vdet2[:,:,i_ep_depth]) /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]; wsink_det2 is a negative number
        else:
            Detcalc1 = 365 * DetCalc1[:,:,i_ep_depth] * 12.01 * Vdet1 /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
            if secdet_path.is_file():
                Detcalc2 = 365 * DetCalc2[:,:,i_ep_depth] * 12.01 * Vdet2 /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
        
        Detcalcct = Detcalc1
        if secdet_path.is_file():
            Detcalcct = Detcalcct + Detcalc2
    
        DetCalc_timeseries = pf.areasum_data(Detcalcct,mesh,mask)
        
        del Detcalc1, Detcalcct
        if secdet_path.is_file():
            del Detcalc2
        
        
        if output:
            self.NPPtotal = NPP_timeseries
            self.PhyTotal = NPPn_timeseries
            self.DiaTotal = NPPd_timeseries
            if cocco_path.is_file():
                self.CoccoTotal = NPPc_timeseries
            if phaeo_path.is_file():
                self.PhaeoTotal = NPPp_timeseries
            self.EPtotal = EP_timeseries
            self.SiEtotal = SiE_timeseries
            self.Calctotal = DetCalc_timeseries
        
        print('TIME-SERIES AVERAGES:')
        print('NPP mean = ',np.nanmean(NPP_timeseries))
        print('NPPd mean = ',np.nanmean(NPPd_timeseries))
        print('NPPn mean = ',np.nanmean(NPPn_timeseries))
        if cocco_path.is_file():
            print('NPPc mean = ',np.nanmean(NPPc_timeseries))
        if phaeo_path.is_file():
            print('NPPp mean = ',np.nanmean(NPPp_timeseries))
        print('EP mean = ',np.nanmean(EP_timeseries))
        print('SiE mean = ',np.nanmean(SiE_timeseries))
        print('CalcE mean = ',np.nanmean(DetCalc_timeseries))
        
        if plotting:
            # plotting total NPP -------------------------------------------------------------------------------        
            fig = plt.figure(figsize=(12,9), facecolor='w', edgecolor='k', tight_layout = True)
            
            plt.subplot(3, 3, 1)
            plt.plot(years,NPP_timeseries,'.-',color='C0',label='Total')
            plt.title('Total NPP')
            plt.ylabel(r'[Pg C yr$^{-1}$]')

            plt.subplot(3, 3, 2)
            plt.plot(years,NPPn_timeseries,'.-g',label='Sphy',color='C1')
            plt.title('small phytoplankton NPP')

            plt.subplot(3, 3, 3)
            plt.plot(years,NPPd_timeseries,'.-r',label='Dia',color='C2')
            plt.title('diatom NPP')
            
            if cocco_path.is_file():
                plt.subplot(3, 3, 4)
                plt.plot(years,NPPc_timeseries,'.-r',label='Cocco',color='C3')
                plt.title('coccolithophore NPP')
            if phaeo_path.is_file():
                plt.subplot(3, 3, 5)
                plt.plot(years,NPPp_timeseries,'.-r',label='Phaeo',color='C4')
                plt.title('phaeocystis NPP')

            # all NPP together
            plt.subplot(3, 3, 6)
            plt.plot(years,NPP_timeseries,'.-',label='Total',color='C0')
            plt.plot(years,NPPn_timeseries,'.-',label='Sphy',color='C1')
            plt.plot(years,NPPd_timeseries,'.-',label='Dia',color='C2')
            if cocco_path.is_file():
                plt.plot(years,NPPc_timeseries,'.-',label='Cocco',color='C3')
            if phaeo_path.is_file():
                plt.plot(years,NPPp_timeseries,'.-',label='Phaeo',color='C4')
            plt.title('NPP')
            plt.ylabel(r'[Pg C yr$^{-1}$]')
            plt.legend(loc='best')

            # Export production
            plt.subplot(3, 3, 7)
            plt.plot(years,EP_timeseries,'.-',color='C3')
            plt.title('EP at 100 m')
            plt.ylabel(r'[Pg C yr$^{-1}$]')

            # Si export
            plt.subplot(3, 3, 8)
            plt.plot(years,SiE_timeseries,'.-',color='C4')
            plt.title('Si export at 100 m')
            plt.ylabel(r'[Tmol Si yr$^{-1}$]')
            
            # CaCO3 export
            plt.subplot(3, 3, 9)
            plt.plot(years,DetCalc_timeseries,'.-',color='C5')
            plt.title('CaCO3 export at 100 m')
            plt.ylabel(r'[Pg C yr$^{-1}$]')
            


            if(savefig):
                plt.savefig(self.savepath+self.runname+'_'+'NPP_timeseries_'+maskstr+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'NPP_timeseries_'+maskstr+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', bbox_inches='tight')
            plt.show(block=False)